In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, ShuffleSplit

properati = pd.read_csv('datos/caba_para_mapa.csv',error_bad_lines=False)

<IPython.core.display.Javascript object>

# RF Clasificador Casas

In [44]:
properati = properati.loc[properati['property_type'] == 'house']
properati.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2751 entries, 4 to 48322
Data columns (total 26 columns):
created_on                    2751 non-null object
property_type                 2751 non-null object
place_name                    2751 non-null object
state_name                    2751 non-null object
lat-lon                       2751 non-null object
lat                           2751 non-null float64
lon                           2751 non-null float64
price                         2751 non-null float64
currency                      2751 non-null object
price_aprox_local_currency    2751 non-null float64
price_aprox_usd               2751 non-null float64
surface_total_in_m2           2751 non-null float64
surface_covered_in_m2         2751 non-null float64
price_usd_per_m2              2751 non-null float64
price_per_m2                  2751 non-null float64
floor                         1 non-null float64
rooms                         1134 non-null float64
expenses         

In [45]:
#paso features con strings a numericos
le_barrio = preprocessing.LabelEncoder()
barrios=properati['place_name']
le_barrio.fit(barrios)
properati['place_name'] = le_barrio.transform(barrios)

le_divisa = preprocessing.LabelEncoder()
divisa=properati['currency']
le_divisa.fit(divisa)
properati['currency'] = le_divisa.transform(divisa)

le_tipo = preprocessing.LabelEncoder()
tipos_prop=properati['property_type']
le_tipo.fit(tipos_prop)
properati['property_type'] = le_tipo.transform(tipos_prop)

### Pruebo con np.histogram

In [46]:
cant_buckets = 15

#buckets
cantidad,rango = np.histogram(properati['price_per_m2'], bins=cant_buckets)
print rango
properati['categories_by_price']=pd.cut(properati['price_per_m2'],rango,labels=np.arange(cant_buckets),include_lowest=True)
properati['price_range']=pd.cut(properati['price_per_m2'],rango)

#lo casteo a float porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 

properati.groupby('categories_by_price').count().sort_values('created_on')

[     0.           6587.26666667  13174.53333333  19761.8         26349.06666667
  32936.33333333  39523.6         46110.86666667  52698.13333333  59285.4
  65872.66666667  72459.93333333  79047.2         85634.46666667
  92221.73333333  98809.        ]


,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,expenses,properati_url,description,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada,price_range
categories_by_price,,,,,,,,,,,,,,,,,,,,,
12.0,8,8,8,8,8,8,8,8,8,8,...,0,8,8,8,8,8,8,8,8,8
10.0,9,9,9,9,9,9,9,9,9,9,...,2,9,9,9,9,9,9,9,9,9
14.0,10,10,10,10,10,10,10,10,10,10,...,0,10,10,10,10,10,10,10,10,10
13.0,13,13,13,13,13,13,13,13,13,13,...,0,13,13,13,13,13,13,13,13,13
11.0,16,16,16,16,16,16,16,16,16,16,...,2,16,16,16,16,16,16,16,16,16
9.0,17,17,17,17,17,17,17,17,17,17,...,3,17,17,17,17,17,17,17,17,17
8.0,66,66,66,66,66,66,66,66,66,66,...,5,66,66,66,66,66,66,66,66,66
7.0,82,82,82,82,82,82,82,82,82,82,...,6,82,82,82,82,82,82,82,82,82
6.0,139,139,139,139,139,139,139,139,139,139,...,5,139,139,139,139,139,139,139,139,139


In [48]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'],properati['dist_a_tren'],properati['dist_a_villa'],\
        properati['dist_a_zona_anegada'],properati['surface_total_in_m2'],properati['place_name'])
y = properati['categories_by_price']

### Pruebo con Quantile . NO MEJORO saltear

In [14]:
from sklearn.preprocessing import QuantileTransformer
scaler = QuantileTransformer(copy=False)

In [15]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['price_usd_per_m2']
print X[0]
X=scaler.fit_transform(X,y)
print X[0]

#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

(7.0, 8.5)
[ 0.86886887  0.86636637]
True
True


### Pruebo con RobustScaler. NO MEJORO saltear

In [16]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler(copy=False)

In [17]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['price_usd_per_m2']
print X[0]
X=scaler.fit_transform(X,y)
print X[0]

#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

(7.0, 8.5)
[ 1.1875      0.97560976]
True
True


### Busco hiper-parametros con random search

In [49]:
%%notify

from sklearn.model_selection import train_test_split
rf = RandomForestClassifier(n_jobs=-1)

oob_score=[True,False]
criterion = ['gini','entropy']
max_features =["auto","log2",None]
n_estimators = np.arange(10,100,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators,"oob_score":oob_score}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=10,n_iter=50) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hyper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 6 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV duro 313.55 segundos para 50 candidatos a hyper-parametros.

Puesto: 1
Promedio training score: 0.995 (std: 0.001)
Promedio validation score: 0.524 (std: 0.036)
Promedio fit time: 0.463s
Hiper-parametros: {'n_estimators': 80, 'max_features': None, 'oob_score': True, 'criterion': 'gini'}

Puesto: 2
Promedio training score: 0.995 (std: 0.001)
Promedio validation score: 0.522 (std: 0.040)
Promedio fit time: 0.246s
Hiper-parametros: {'n_estimators': 40, 'max_features': None, 'oob_score': True, 'criterion': 'gini'}

Puesto: 3
Promedio training score: 0.995 (std: 0.001)
Promedio validation score: 0.521 (std: 0.040)
Promedio fit time: 0.466s
Hiper-parametros: {'n_estimators': 40, 'max_features': None, 'oob_score': True, 'criterion': 'entropy'}

Puesto: 3
Promedio training score: 0.995 (std: 0.001)
Promedio validation score: 0.521 (std: 0.030)
Promedio fit time: 0.584s
Hiper-parametros: {'n_estimators': 60, 'max_features': None, 'oob_score': False, 'criterion': 'entropy'}

Puesto:

<IPython.core.display.Javascript object>

In [50]:
mejor_rf = search.best_estimator_
print mejor_rf.feature_importances_ # la importancia de cada feature (cuanta informacion le provee al rf) ordenado tal y cual esta zipeado el X

[ 0.09324687  0.09853616  0.15114616  0.11534589  0.13726193  0.33876554
  0.06569745]


In [51]:
search.score(X_test,y_test) # pruebo el estimador refiteado con el set de test

0.57168784029038111